## Assignment: Segmentation and Clustering Neighbourhood in Toronto

In [49]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup

In [50]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data,'html5lib')

In [51]:
tag_object = soup.ta

In [52]:
table_contents=[]
table=soup.find('table')
table
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


In [53]:
# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [54]:
df.shape

(103, 3)

In [55]:
# !conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

In [56]:
lat_lng = pd.read_csv('Geospatial_Coordinates.csv')
lat_lng.rename(columns ={'Postal Code':'PostalCode'}, inplace = True)
lat_lng.head()

# initialize your variable to None
# lat_lng_coords = None
# postal_code = df['PostalCode']
# postal_code
# for  x, pin in postal_code.items():
# # loop until you get the coordinates
#     while(lat_lng_coords is None):
#       g = geocoder.arcgis('{}, Toronto, Ontario'.format(pin))
#       lat_lng_coords = g.latlng
#     # print(pin)
#     latitude = lat_lng_coords[0]
#     longitude = lat_lng_coords[1]
#     lat_lng = lat_lng.append({'Latitude':latitude,'Longitude':longitude}, ignore_index = True)
    
# lat_lng['Latitude'].value_counts()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [57]:

df = df.merge(lat_lng, on = ['PostalCode'],how = 'left')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
